In [2]:
import pandas as pd
import category_encoders as ce
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
# read in data:
df = pd.read_csv('adoption_use (1).csv', index_col=None)
df = df.drop(columns='Unnamed: 0')
df.reset_index(drop=True)

,animal_type,breed,color,outcome,season_arrived,year_arrived,age,fixed_Fixed,gender
0,Dog,German Shepherd Mix,Black/Brown,Not adopted,Fall,2019,0.25,0,Female
1,Dog,German Shepherd Mix,Black/Brown,Not adopted,Fall,2019,0.25,0,Female
2,Dog,Chihuahua Shorthair,Tricolor,Not adopted,Fall,2019,1.00,1,Male
3,Dog,Labrador Retriever Mix,Brown/Black,Not adopted,Fall,2019,6.00,1,Male
4,Dog,Labrador Retriever Mix,Black/White,Not adopted,Fall,2019,2.00,1,Male
...,...,...,...,...,...,...,...,...,...
59523,Cat,Domestic Shorthair Mix,Blue Tabby,Not adopted,Summer,2014,0.25,0,Male
59524,Cat,Domestic Shorthair Mix,Blue Tabby,Not adopted,Summer,2014,0.25,0,Male
59525,Cat,Domestic Shorthair Mix,Black,Not adopted,Summer,2014,0.25,0,Male
59526,Cat,Domestic Shorthair Mix,Black,Not adopted,Summer,2014,0.25,0,Male


In [3]:
# 3-way split:
test = df[(df['year_arrived'] == 2019)]  
val =  df[(df['year_arrived']== 2018)]  
train = df[(df['year_arrived'] < 2018)]
test = test.drop(columns='year_arrived')
val = val.drop(columns='year_arrived')
train = train.drop(columns='year_arrived')

In [4]:
# set X features matrices and y target vectors:
target = 'outcome'
X_train = train.drop(columns=target)
y_train = train[target]
X_val = val.drop(columns=target)
y_val = val[target]
X_test = test.drop(columns=target)
y_test = test[target]


In [6]:
# pipeline after hyperparameter optimization:
mypipeline = make_pipeline(
    ce.OrdinalEncoder(), 
    RandomForestClassifier(n_estimators=479, max_depth=5, 
                               max_features=0.12718863383484313, random_state=42, 
                               n_jobs=-1)
)
# Fit on train, score on val
mypipeline.fit(X_train, y_train)
print('Validation Accuracy', mypipeline.score(X_val, y_val))

Validation Accuracy 0.86345300524405


In [7]:
from joblib import dump
dump(mypipeline, 'mypipeline.joblib')

['mypipeline.joblib']

In [8]:
def predict(animal_type, gender, fixed_Fixed, age, color, breed, season_arrived):
    df = pd.DataFrame(
        columns=['animal_type', 'gender', 'fixed_Fixed', 'age', 'color', 'breed', 'season_arrived'],
        data=[[animal_type, gender, fixed_Fixed, age, color, breed, season_arrived]]
    )
    y_pred=thepipeline.predict(df)[0]
    if y_pred == 'Adopted':
        y_pred_proba = thepipeline.predict_proba(df)[0][0]
        return f'{y_pred_proba*100:.0f}% chance of {y_pred}'
    else:
        y_pred_proba = thepipeline.predict_proba(df)[0][1]
        return f'{y_pred_proba*100:.0f}% chance of {y_pred}'